# SURGE Demo

Here, we simulate some data eQTL data (where the eQTL effect sizes change as function context) and assess SURGE's ability to re-capture the uknown contexts.

This notebook should also provide an example of how SURGE can be easily run within python.

In [1]:
import numpy as np
import surge.surge_inference
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

## Simulate some data

To assess SURGE's ability to accurately capture contexts underlying context-specific eQTLs, we perform the following simulation experiment:

We randomly generated genotype and expression matrices across $T$ variant-gene-pairs and $N$ RNA samples. For each simulated variant-gene pair, we simulated the genotype vector ($G$) across the $N$ samples according to the following probability distributions:

$af_n \sim Uniform(.1,.9)$

$G_n \sim Binomial(2, af_n)$

Then we standardized genotype to have mean 0 and variance 1. We call the standardized genotype in the nth sample $G^*_n$

Next, we simulated the expression vector (y) across the $N$ samples using the following probability distributions (conditional on the simulated standardized genotype):

$y_n \sim N(\mu + \beta G^*_n +\sum_k G^*_n *U_{nk} V_k \theta_k, 1)$

$\mu \sim N(0,1)$

$\beta \sim N(0,1)$

$U_{nk} \sim N(0,1)$

$V_k \sim N(0, \gamma)$

$\theta_k \sim Bernoulli(p)$



This simulation, therefor, evaluates SURGE's ability to re-capture the simulated latent contexts (U) as a function of the simulation hyper-parameters:
- The number of latent contexts (K; num_components)
- The sample size (N; num_samples)
- The strength of the interaction terms ($\gamma$, t_statistic)
- The fraction of tests that are context-specific eQTLs for a particular context ($p$; missingness_fraction)




In [2]:
# Generate genotype vector for a particular snp
def get_genotype_vec(num_samples, af):
    allele1 = np.random.binomial(n=1, p=af,size=num_samples)
    allele2 = np.random.binomial(n=1, p=af,size=num_samples)
    genotype_vec = allele1 + allele2
    return genotype_vec


def standardize_variance_ratio_between_expression_and_genotype(Y, G):
    num_tests = Y.shape[1]
    for test_num in range(num_tests):
        test_sdev = np.std(Y[:, test_num]/G[:,test_num])
        G[:, test_num] = G[:, test_num]*test_sdev
    G = G/np.std(G)
    return G

# Generate gene expression data (Y), and genotype data (G)
def generate_eqtl_factorization_data_with_no_sample_repeat(num_samples, num_tests, num_components, t_statistic, missingness_fraction):
    # Simulate genotype
    G = np.zeros((num_samples, num_tests))
    G_raw = np.zeros((num_samples, num_tests))
    for test_num in range(num_tests):
        af = np.random.uniform(.1,.9)
        # Get genotype for this particular snp
        genotype_vec = get_genotype_vec(num_samples, af)
        G_raw[:, test_num] = genotype_vec
        # Standardized genotype
        G[:, test_num] = (genotype_vec - np.mean(genotype_vec))/np.std(genotype_vec)

    # Simulate U
    U = np.random.standard_normal(size=(num_samples, num_components))
    for component_num in range(num_components):
        U[:, component_num] = (U[:, component_num] - np.mean(U[:, component_num]))/np.std(U[:, component_num])
    
    # Simulate V
    V = np.random.normal(loc=0.0, scale=t_statistic, size=(num_components, num_tests))
    # Add missingness to V
    for test_num in range(num_tests):
        for component_num in range(num_components):
            V[component_num, test_num] = V[component_num, test_num]*np.random.binomial(n=1,p=missingness_fraction)

    # Get Expected expression value of each (sample, gene) pair
    predicted_mean = np.zeros((num_samples, num_tests))  + G*np.dot(U,V)
    # Simulate genotype fixed effect size
    betas = []
    Y = np.zeros((num_samples, num_tests))
    for test_num in range(num_tests):
        beta = np.random.normal(loc=0.0, scale=.1)
        predicted_mean[:, test_num] = predicted_mean[:, test_num] + G[:, test_num]*beta
        Y[:, test_num] = np.random.normal(predicted_mean[:,test_num])
        betas.append(beta)
    
    # Standardized Y
    for test_num in range(num_tests):
        Y[:, test_num] = (Y[:, test_num] - np.mean(Y[:, test_num]))/np.std(Y[:, test_num])
    
    # Scale genotype vector for each test by the standard deviation of Y/G
    # This scaling encourages the low-dimensional factorization (UV) to explain variance equally across tests
    G = standardize_variance_ratio_between_expression_and_genotype(Y, G)
    
    return Y, G, np.asarray(betas), U, V


In [3]:
# Simulation parameters
num_samples = 250
num_tests=1000
t_statistic = .75
missingness_fraction = .3
simulated_factor = 5
np.random.seed(2)

# Simulate the data
Y, G, betas, U_sim, V_sim = generate_eqtl_factorization_data_with_no_sample_repeat(num_samples, num_tests, simulated_factor, t_statistic, missingness_fraction)



## Run SURGE

At this point we have successfully simulated eQTL data. Notably we have generated an standardized expression matrix ($Y$) of dimension $N$X$T$ and we have generated a standardized genotype matrix ($G^*$) of dimension $N$X$T$. The $t^{th}$ column of $Y$ reflects the gene corresponding to the $t^{th}$ variant-gene pair. The $t^{th}$ column of $G$ reflects the variant corresponding to the $t^{th}$ variant-gene pair. Notably, each column of $Y$ has mean 0 and stardard deviation 1. Additionally, each column of $G$ has mean 0 and is scaled by the standard deviation of Y[:, t]/G[:, t]. This scaling encourages the low-dimensional factorization (UV) to explain variance equally across tests.

Using this simulated data, we can run SURGE to re-discover the simulated contexts.

It is important to note that this simulated expression was not affected by covariates nor was simulated expression affected by sample repeat structure . Therefore, we ran we ran SURGE:
- We did not model the effects of covariates on gene expression in SURGE. However, covariates could be easily included by changing cov into a matrix that contains a column for each covariate (as well as column corresponding to the intercept). 
- We did not model the effects of sample repeat structure on gene expression in SURGE. However, sample repeat structure could easily be included by setting `re_boolean=True`, and by including `z=sample_repeat` as an arguement in `fit`. `sample_repeat` is a vector of length $N$ where each element is an integer corresponding to the individual that sample came from.

In [ ]:
# Covariate matrix is just an intercept as there are no simulated covariates
cov = np.ones((num_samples, 1))

# Create SURGE object (w/ no sample repeat structure): 
surge_obj = surge.surge_inference.SURGE_VI(K=10, re_boolean=False)
surge_obj.fit(G=G, Y=Y, cov=cov)


*********************************************************
SURGE
Single cell Unsupervised Regulation of Gene Expression
*********************************************************
###############################
Initialize variables
###############################
250 samples detected
1000 tests detected
10 latent factors detected
###############################
Begin Coordinate ascent variational inference (CAVI) iterative algorithm
###############################
Variational Inference iteration: 0
delta ELBO: 129674150.70252755
Variational Inference iteration: 1
delta ELBO: 10254.893435208302
Variational Inference iteration: 2
delta ELBO: 10361.034078769677
Variational Inference iteration: 3
delta ELBO: 15470.044559486385
Variational Inference iteration: 4
delta ELBO: 11611.206279800623
Variational Inference iteration: 5
delta ELBO: 5047.356142696401
Variational Inference iteration: 6
delta ELBO: 1008.8867376417038
Variational Inference iteration: 7
delta ELBO: 539.8357458042447
Variati

## Visualize learned SURGE latent contexts

SURGE optimization has converged. We can now visualize the results. 

First, we can examine the ELBO over variational iterations.

In [ ]:
# check to make sure ELBO is monotonically increasing (Note: ELBO is gauranteed to monotonically increase)
plt.plot(surge_obj.elbo[1:]);  # Skipped first iteration in visualization b/c it is such a large change that it completely dominates the plot
plt.xlabel('Iteration');
plt.ylabel('ELBO');

Plot PVE by each of SURGE latent contexts

In [ ]:
# Order SURGE contexts by PVE
context_order = np.argsort(-surge_obj.factor_pve)
# Plot PVE for each of latent contexts
plt.plot(surge_obj.factor_pve[context_order], '-ok');
plt.xlabel('Learned SURGE latent context');
plt.ylabel('PVE');

5 learned SURGE latent contenxts have non-zero PVE, whereas the other 5 SURGE latent contexts have approximately 0 PVE. This is expected given we simulated 5 latent contexts. 

Next, plot absolute correlation between learned SURGE latent contexts and simulated SURGE latent contexts. Here, we filter out non-informative learned latent contexts (ie those with PVE < 1e-4).

In [ ]:
# Extact learned SURGE latent contexts (ordered by PVE)
U_learned = surge_obj.U_mu[:, context_order]
# Filter out contexts with PVE < 1e-4
context_pves = surge_obj.factor_pve[context_order]
U_learned = U_learned[:, context_pves >= 1e-4]

# Compute correlations
K_sim = U_sim.shape[1]
K_learned = U_learned.shape[1]
corr_mat = np.zeros((K_sim, K_learned))

for k_sim in range(K_sim):
    for k_learned in range(K_learned):
        corr_mat[k_sim, k_learned] = np.corrcoef(U_sim[:, k_sim], U_learned[:, k_learned])[0,1]

sns.heatmap(np.abs(corr_mat));
plt.xlabel('Learned SURGE latent contexts');
plt.ylabel('Simulated SURGE latent contexts');


As can be seen in the above heatmap, the learned SURGE latent contexts are well correlated with the simulated SURGE latent contexts. However, some of the simulated SURGE latent contexts are explained by multiple learned SURGE latent contexts. For example, learned SURGE latent context 1 is correlated (nearly equally) with simulated SURGE context 1 and 2. This result is unsuprising and is well-documented by dimensionality reduction theory: solutions are equivalent up to a rotation.

Another way to view how well the learned SURGE latent contexts capture the simulated SURGE latent contexts is to ask of much of the variance in a simulated SURGE latent context is explained by all of the SURGE latent contexts:

In [ ]:
# Calculate R-squared in each simulated component
r_squared = []
for k_sim in range(K_sim):
    reg = LinearRegression().fit(U_learned, U_sim[:,k_sim])
    r_squared.append(reg.score(U_learned, U_sim[:,k_sim]))

# Plot
plt.plot(r_squared, '-ok');
plt.xlabel('Simulated SURGE latent context');
plt.ylabel('R^2');

## Extract parameters from fitted SURGE object

Now, we will go over some useful parameters to extract from the fitted SURGE object.


### 1. Expected value of SURGE latent contexts.

    This is a matrix of dimensions NXK where N is the number of samples and K is the number of latent contexts. An element of this matrix represents the estimated expected value of the SURGE latent context for a particular latent context for a particular sample.


In [ ]:
print(surge_obj.U_mu)

### 2. Variance of SURGE latent contexts
   
    This a matrix of dimensions NXK where N is the number of samples and K is the number of latent contexts. An element of this matrix represents the estimated variance of the expected value of the SURGE latent contexts for a particular latent context for a particular sample.
   

In [ ]:
print(surge_obj.U_var)

### 3. ELBO
   
    This a vector of length number of variational iterations where each element represents that elbo at each iteration. Note, this vector is gauranteed to be monotonically increasing.

In [ ]:
print(surge_obj.elbo)

### 3. Percent variance explained of each of the SURGE latent contexts
   
    This is a vector of length number of latent contexts. Each element of this vector represents the PVE (Based on bottom of P21 of https://arxiv.org/pdf/1802.06931.pdf) of each of the SURGE latent contexts. Generally we filter out contexts based on those we PVE < 1e-4.

In [ ]:
print(surge_obj.factor_pve)

### 4. Expected value of latent context effect sizes
   
    This is a matrix of dimensions KXT where T is the number of eqtl tests and K is the number of latent contexts. An element of this matrix represents the estimated interaction effect size corresponding to the latent context and eqtl of the element.

In [ ]:
print(surge_obj.V_mu)

### 5. Variance of latent context effect sizes
   
    This is a matrix of dimensions KXT where T is the number of eqtl tests and K is the number of latent contexts. An element of this matrix represents the estimated interaction effect size variance corresponding to the latent context and eqtl of the element.

In [ ]:
print(surge_obj.V_var)